<a href="https://colab.research.google.com/github/Dr-Sanjay/Assignment_Sanjay/blob/main/Placement%20Assignment_(Sanjay)/Deep_Learning_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep_Learning_Solutions

Question 1 -
Implement 3 different CNN architectures with a comparison table for the MNSIT
dataset using the Tensorflow library.
Note -
1. The model parameters for each architecture should not be more than 8000
parameters
2. Code comments should be given for proper code understanding.
3. The minimum accuracy for each accuracy should be at least 96%

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Add a channels dimension (MNIST images are grayscale)
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define a function to build and compile a CNN model
def build_cnn_model():
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# Define the first CNN architecture
model_1 = build_cnn_model()
model_1.summary()

# Train the model
history_1 = model_1.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
_, accuracy_1 = model_1.evaluate(x_test, y_test)
print("Accuracy of Model 1:", accuracy_1)

# Define the second CNN architecture
model_2 = tf.keras.Sequential([
    layers.Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])
model_2.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_2.summary()

# Train the model
history_2 = model_2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
_, accuracy_2 = model_2.evaluate(x_test, y_test)
print("Accuracy of Model 2:", accuracy_2)

# Define the third CNN architecture
model_3 = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")
])
model_3.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_3.summary()

# Train the model
history_3 = model_3.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
_, accuracy_3 = model_3.evaluate(x_test, y_test)
print("Accuracy of Model 3:", accuracy_3)


11490434/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 64)                346176    
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 347,146
Trainable params: 347,146
Non-train

Question 2 -
Implement 5 different CNN architectures with a comparison table for CIFAR 10
dataset using the PyTorch library
Note -
1. The model parameters for each architecture should not be more than 10000
parameters
2 Code comments should be given for proper code understanding

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the transformations for data preprocessing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Set batch size and number of workers for data loaders
batch_size = 128
num_workers = 2

# Create data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, running_loss / len(trainloader)))

# Function to test the model
def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Architecture definition
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5)
        self.fc1 = nn.Linear(128 * 5 * 5, 1024)  # Adjusted input size
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# Model 2: LeNet-5
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # Architecture definition
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x



# Model 3: VGG-like Architecture
class VGGLike(nn.Module):
    def __init__(self):
        super(VGGLike, self).__init__()
        # Architecture definition
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512*4*4, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Model 4: ResNet-like Architecture
class ResNetLike(nn.Module):
    def __init__(self):
        super(ResNetLike, self).__init__()
        # Architecture definition
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.residual1 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
        )
        self.residual2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
        )
        self.fc = nn.Linear(16*32*32, 10)

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.relu(x)
        x = self.residual1(x) + residual
        residual = x
        x = self.relu(x)
        x = self.residual2(x) + residual
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Model 5: Custom Architecture
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Architecture definition
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64*8*8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Create instances of the models
simple_cnn = SimpleCNN()
lenet5 = LeNet5()
vgg_like = VGGLike()
resnet_like = ResNetLike()
custom_cnn = CustomCNN()

# Print the number of parameters for each model
print('Number of Parameters:')
print('Simple CNN: ', sum(p.numel() for p in simple_cnn.parameters()))
print('LeNet-5: ', sum(p.numel() for p in lenet5.parameters()))
print('VGG-like: ', sum(p.numel() for p in vgg_like.parameters()))
print('ResNet-like: ', sum(p.numel() for p in resnet_like.parameters()))
print('Custom CNN: ', sum(p.numel() for p in custom_cnn.parameters()))

# Train and test the models
criterion = nn.CrossEntropyLoss()

# Define the optimizers and learning rates for each model
optimizer_simple_cnn = optim.Adam(simple_cnn.parameters(), lr=0.001)
optimizer_lenet5 = optim.Adam(lenet5.parameters(), lr=0.001)
optimizer_vgg_like = optim.Adam(vgg_like.parameters(), lr=0.001)
optimizer_resnet_like = optim.Adam(resnet_like.parameters(), lr=0.001)
optimizer_custom_cnn = optim.Adam(custom_cnn.parameters(), lr=0.001)

# Train the models
def train_model(model, criterion, optimizer):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if i % 100 == 99:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

# Create instances of the models
simple_cnn = SimpleCNN()
lenet5 = LeNet5()
vgg_like = VGGLike()
resnet_like = ResNetLike()
custom_cnn = CustomCNN()

# Define the learning rate
learning_rate = 0.001

# Define the optimizers and learning rates for each model
optimizer_simple_cnn = optim.Adam(simple_cnn.parameters(), lr=learning_rate)
optimizer_lenet5 = optim.Adam(lenet5.parameters(), lr=learning_rate)
optimizer_vgg_like = optim.Adam(vgg_like.parameters(), lr=learning_rate)
optimizer_resnet_like = optim.Adam(resnet_like.parameters(), lr=learning_rate)
optimizer_custom_cnn = optim.Adam(custom_cnn.parameters(), lr=learning_rate)

# Define the number of epochs
num_epochs = 10

# Import the necessary libraries
import torchvision
import torchvision.transforms as transforms

# Define the transforms to apply to the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the CIFAR-10 training dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Create the data loader for training
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


# Train the models
train_model(simple_cnn, criterion, optimizer_simple_cnn)
train_model(lenet5, criterion, optimizer_lenet5)
train_model(vgg_like, criterion, optimizer_vgg_like)
train_model(resnet_like, criterion, optimizer_resnet_like)
train_model(custom_cnn, criterion, optimizer_custom_cnn)

# Test the models and print the accuracy
accuracy_simple_cnn = test_model(simple_cnn)
accuracy_lenet5 = test_model(lenet5)
accuracy_vgg_like = test_model(vgg_like)
accuracy_resnet_like = test_model(resnet_like)
accuracy_custom_cnn = test_model(custom_cnn)

# Print the accuracy for each model
print('\nAccuracy:')
print('Simple CNN: {:.2f}%'.format(accuracy_simple_cnn))
print('LeNet-5: {:.2f}%'.format(accuracy_lenet5))
print('VGG-like: {:.2f}%'.format(accuracy_vgg_like))
print('ResNet-like: {:.2f}%'.format(accuracy_resnet_like))
print('Custom CNN: {:.2f}%'.format(accuracy_custom_cnn))



Question 3 -
Train a Pure CNN with less than 10000 trainable parameters using the MNIST
Dataset having minimum validation accuracy of 99.40%
Note -
1. Code comments should be given for proper code understanding.
2. Implement in both PyTorch and Tensorflow respectively

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(7 * 7 * 32, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 7 * 7 * 32)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

# Initialize the model
model = SimpleCNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100.0 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    validation_accuracy = 100.0 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {validation_accuracy:.2f}%")
    
    # Check if the validation accuracy criterion is met
    if validation_accuracy >= 99.40:
        break

# Save the trained model
torch.save(model.state_dict(), 'simple_cnn.pth')


Question 4 -
Design an end-to-end solution with diagrams for object detection use cases
leveraging AWS cloud services and open-source tech
Note -
1. You need to use both AWS cloud services and open-source tech to design the
entire solution
2. The pipeline should consist of a data pipeline, ml pipeline, deployment pipeline,
and inference pipeline.
3. In the data pipeline, you would be designing how to get the data from external or
existing sources and tech used for the same
4. In the ml pipeline, you would be designing how to train the model, and what all
algorithms, techniques, etc. would you be using. Again, tech used for the same 5.

Since this is a deep learning project, the use of GPUs, and how effectively are you
using them to optimize for cost and training time should also be taken into
consideration.
6. In the deployment pipeline, you would be designing how effectively and
efficiently you are deploying the model in the cloud,
7. In the inference pipeline, consider the cost of inference and its optimization

related to computing resources and handling external traffic
8. You can use any tool to design the architecture
9. Do mention the pros and cons of your architecture and how much further it can
be optimized and its tradeoffs.
10. Do include a retraining approach as well.
11. Try to include managed AWS resources for deep learning like AWS Textract,
AWS Sagemaker, etc., and not just general-purpose compute resources like S3,
EC2, etc. Try to mix the best of both services

Question 5 -

In Question 4, you have designed the architecture for an object detection use case
leveraging AWS Cloud, similarly, here you will be designing for Document
Classification use case leveraging Azure Cloud services.
Note -
1. Most of the points are the same as in Question 4, just cloud services will
change